In [2]:
import re
import subprocess
import time
import os
from random import randrange 
import glob 


file = ""
filename = f"run_script.sh"

with open(filename) as f:
    file = f.read()


batch_sizes = [512, 256]
lrs = [1e-2]
hidden_channelss = [32]
num_layerss = [2]

for lr in lrs:
    for batch_size in batch_sizes:
        for hidden_channels in hidden_channelss:
            for num_layers in num_layerss:
                exp_name = f"coco-{hidden_channels}-{num_layers}-{batch_size}-{lr}"

                result_dir = f"results/{exp_name}"
                # if not os.path.exists(result_dir):
                #     os.makedirs(result_dir)

                file = re.sub('--lr .*', f"--lr {lr} " + r"\\" , file)
                file = re.sub('--batch_size .*', f"--batch_size {batch_size} " + r"\\" , file)
                file = re.sub('--hidden_channels .*', f"--hidden_channels {hidden_channels} " + r"\\" , file)
                file = re.sub('--num_layers .*', f"--num_layers {num_layers} " + r"\\" , file)
                file = re.sub('--exp_name .*', f"--exp_name {exp_name} " + r"\\" , file)

                file = re.sub('--job-name=.*', f"--job-name={batch_size}{lr}", file)
                file = re.sub('--output .*', f"--output {result_dir}.log", file)
                file = re.sub('--error .*', f"--error {result_dir}.log", file)

                # if qscore_reg and "--qscore_reg \\" not in file:
                #     file += "\n--qscore_reg \\"
                # elif qscore_reg == False and "--qscore_reg \\" in file:
                #     split = file.split("\n--qscore_reg \\")
                #     file = "".join(split)

                print(file)

                with open(filename, 'w') as f:
                    f.write(file)

                subprocess.run(["sbatch", filename])

#!/bin/bash
#SBATCH --job-name=5120.01
#SBATCH --output results/coco-32-2-512-0.01.log
#SBATCH --error results/coco-32-2-512-0.01.log
#SBATCH --time=72:00:00
#SBATCH --gres=gpu:rtxa5000:1
#SBATCH --qos=scavenger
#SBATCH --account=scavenger
#SBATCH --partition=scavenger
#SBATCH --cpus-per-task=16
#SBATCH --mem=32G

python3 train.py \
--vision_graph_data coco_visual_graph \
--text_graph_data coco_text_graph \
--batch_size 512 \
--hidden_channels 32 \
--num_layers 2 \
--lr 0.01 \
--exp_name coco-32-2-512-0.01 \


In [1]:
import subprocess
for i in range(2332000, 2332021):
    subprocess.run(["scancel", str(i)])